# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [37]:
# Use this cell to write your code for Task 1

import numpy as np
import pandas as pd

city = pd.read_csv("house_sales.csv")
unknown_city = city[city['city'] == "--"]
missing_city = len(unknown_city)
missing_city

73

# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [38]:
# Use this cell to write your code for Task 2

df = pd.read_csv("house_sales.csv")

# Identifying and handling missing values and string manipulation
df["city"] = df["city"].str.replace('--', 'Unknown')
df['city'] = df['city'].fillna('Unknown')
df = df.dropna(subset=['sale_price'])
df['sale_date'] = df['sale_date'].fillna('2023-01-01')
mean_months_listed = round(df['months_listed'].mean(), 1)
df['months_listed'] = df['months_listed'].fillna(mean_months_listed)
mean_bedrooms = round(df['bedrooms'].mean())
df['bedrooms'] = df['bedrooms'].fillna(mean_bedrooms)
house_type_mapping = {
    'Det.': 'Detached',
    'Detached': 'Detached',
    'Semi': 'Semi-detached',
    'Semi-detached': 'Semi-detached',
    'Terr.': 'Terraced',
    'Terraced': 'Terraced'
}
df['house_type'] = df['house_type'].map(house_type_mapping)
most_common_house_type = df['house_type'].mode()[0]
df['house_type'] = df['house_type'].fillna(most_common_house_type)
df["area"] = df["area"].str.replace(' sq.m.', '', regex=False)
df["area"] = pd.to_numeric(df["area"], errors="coerce")
mean_area = round(df['area'].mean(), 1)
df['area'] = df['area'].fillna(mean_area)


# Converting/ensuring that column values are of appropriate data type
#df['house_id'] = df['house_id'].astype(str)
#df["city"] = df["city"].astype('category')
df['sale_date'] = pd.to_datetime(df['sale_date'])
df['months_listed'] = pd.to_numeric(df['months_listed'])
df['bedrooms'] = pd.to_numeric(df['bedrooms'])
ordered_cat = ['Terraced', 'Semi-detached', 'Detached']
df['house_type'] = pd.Categorical(df['house_type'], categories=ordered_cat, ordered=True)
df['area'] = pd.to_numeric(df['area'])

clean_data = df
clean_data.head()

,house_id,city,sale_price,sale_date,months_listed,bedrooms,house_type,area
0,1217792,Silvertown,55943,2021-09-12,5.4,2,Semi-detached,107.8
1,1900913,Silvertown,384677,2021-01-17,6.3,5,Detached,498.8
2,1174927,Riverford,281707,2021-11-10,6.9,6,Detached,542.5
3,1773666,Silvertown,373251,2020-04-13,6.1,6,Detached,528.4
4,1258487,Silvertown,328885,2020-09-24,8.7,5,Detached,477.1


In [39]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   house_id       1500 non-null   int64         
 1   city           1500 non-null   object        
 2   sale_price     1500 non-null   int64         
 3   sale_date      1500 non-null   datetime64[ns]
 4   months_listed  1500 non-null   float64       
 5   bedrooms       1500 non-null   int64         
 6   house_type     1500 non-null   category      
 7   area           1500 non-null   float64       
dtypes: category(1), datetime64[ns](1), float64(2), int64(3), object(1)
memory usage: 83.8+ KB


# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [40]:
# Use this cell to write your code for Task 3

df =  pd.read_csv('house_sales.csv')

price_by_rooms = df.groupby('bedrooms').agg(
    avg_price=('sale_price', 'mean'),
    var_price=('sale_price', 'var')).reset_index()

# Rounding the results to 2 decimal places
price_by_rooms['avg_price'] = price_by_rooms['avg_price'].round(1)
price_by_rooms['var_price'] = price_by_rooms['var_price'].round(1)
price_by_rooms

,bedrooms,avg_price,var_price
0,2,67076.4,5.652896e+08
1,3,154665.1,2.378289e+09
2,4,234704.6,1.725211e+09
3,5,301515.9,2.484328e+09
4,6,375741.3,3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [41]:
# Use this cell to write your code for Task 4

from sklearn.linear_model import LinearRegression

train_data = pd.read_csv("train.csv")
validation_data = pd.read_csv("validation.csv")

train_data = pd.get_dummies(train_data, columns=['city', 'house_type'])
validation_data = pd.get_dummies(validation_data, columns=['city', 'house_type'])

X_train = train_data.drop(["sale_price", "house_id", "sale_date"], axis=1)
y_train = train_data["sale_price"]  

X_val = validation_data.drop(["house_id", "sale_date"], axis=1)

lr = LinearRegression()
lr.fit(X_train, y_train)

lr_predicted_prices = lr.predict(X_val)

base_result = pd.DataFrame({
    'house_id': validation_data['house_id'],
    'price': lr_predicted_prices})
base_result

,house_id,price
0,1331375,121527.827316
1,1630115,304386.625267
2,1645745,384760.100656
3,1336775,123976.268985
4,1888274,271186.199353
...,...,...
295,1986255,351916.468218
296,1896276,370138.966354
297,1758223,259024.117384
298,1752010,169120.160936


In [42]:
mae = mean_absolute_error(y_train, lr.predict(X_train))
mse = mean_squared_error(y_train, lr.predict(X_train))
rmse = np.sqrt(mse)
r2 = r2_score(y_train, lr.predict(X_train))

mae, mse

(16398.833938691972, 463704535.7273433)

In [43]:
rmse, r2

(21533.799844136734, 0.9662723112446464)

# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [44]:
# Use this cell to write your code for Task 5

from sklearn.ensemble import RandomForestRegressor

train_data = pd.read_csv("train.csv")
validation_data = pd.read_csv("validation.csv")

train_data = pd.get_dummies(train_data, columns=['city', 'house_type'])
validation_data = pd.get_dummies(validation_data, columns=['city', 'house_type'])

X_train = train_data.drop(["sale_price", "house_id", "sale_date"], axis=1)
y_train = train_data["sale_price"]  

X_val = validation_data.drop(["house_id", "sale_date"], axis=1)

rf = RandomForestRegressor(n_estimators=50, random_state=42)
rf.fit(X_train, y_train)

rf_predicted_prices = rf.predict(X_val)

compare_result = pd.DataFrame({
    'house_id': validation_data['house_id'],
    'price': rf_predicted_prices})

compare_result

,house_id,price
0,1331375,82653.74
1,1630115,300895.88
2,1645745,405169.26
3,1336775,107466.78
4,1888274,269405.22
...,...,...
295,1986255,365031.48
296,1896276,393454.00
297,1758223,264763.34
298,1752010,176455.52


In [45]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_train, rf.predict(X_train))
mse = mean_squared_error(y_train, rf.predict(X_train))
rmse = np.sqrt(mse)
r2 = r2_score(y_train, rf.predict(X_train))

mae, mse

(3976.1071388888895, 33717477.450952545)

In [46]:
rmse, r2

(5806.675249310275, 0.9975475491450658)